In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

# Area de estudio

Debido al tamaño del stack de bandas es probable que el tamaño del area de estudio sea demasiado grande para exportar los datos, por lo que una alternativa es seleccionar puntos aleatorios dentro de la misma y generar un buffer sobre ellos para obetner una muestra representativa.

In [2]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/facub-gee/assets/Nepal')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
# Añadir Google Satellite como mapa base
Map.add_basemap('SATELLITE')
Map.centerObject(roi)


In [3]:
# Generar 10 puntos aleatorios dentro del polígono
random_points = ee.FeatureCollection.randomPoints(roi, 5)

# Crear un buffer de 10 km (10000 metros) alrededor de cada punto
buffered_points = random_points.map(lambda feature: feature.buffer(10000))

# Función para añadir las geometrías al mapa
def add_geometries_to_map(geometries, style, name):
    Map.addLayer(geometries, style, name)

# Añadir los puntos aleatorios al mapa
add_geometries_to_map(random_points, {}, 'Random Points')

# Añadir los buffers al mapa
add_geometries_to_map(buffered_points, {'color': 'red'}, 'Buffered Points (10 km)')

roi = buffered_points
# Mostrar el mapa
Map

Map(center=[26.89483839719102, 86.09164999611022], controls=(WidgetControl(options=['position', 'transparent_b…

# Llamado a las colecciones 

In [4]:
# Parámetros de visualización para imágenes multiespectrales en falso color nir-swir1-swir2
visFalseColor = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B11', 'B12']
}

# Parámetros de visualización para la biomasa
visBiomass = {'min': 0, 'max': 100, 'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']}

# Parámetros de visualización para Sentinel-1
visParamsS1 = {
    'min': -25,
    'max': 0,
    'bands': ['VV', 'VH']
}

# Definir el año de análisis
year = 2020

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

# Optico

In [12]:
# Filtrar la colección Sentinel-2
s2Col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Filtrar la colección Sentinel-2 cloud score
cloudCol = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Combinar la colección, máscara de nubes y hacer un composite mediano
def mask_clouds(image):
    cloud_score = ee.Image(cloudCol.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

image = s2Col.map(mask_clouds).median().toFloat().clip(roi)

# Calcular NDVI y MNDWI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
# Calcular NDBI
ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')

# Añadir bandas NDVI, MNDWI y NDBI a la imagen
image = image.addBands(ndvi).addBands(mndwi).addBands(ndbi)

# Integrar la banda "label" de Dynamic World
dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dwLabel = dwCol.select('label').median().clip(roi)


# Añadir la banda "label" de Dynamic World a la imagen
image = image.addBands(dwLabel.rename('label'))

# Cargar la imagen de altura de dosel
canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")

# Recortar la imagen de altura de dosel a la ROI
canopy_height = canopy_height.clip(roi)

# Agregar la banda de altura de dosel a la imagen de datos de biomasa de GEDI
image = image.addBands(canopy_height.rename('CH'))
# Aplicar la máscara a la imagen original


# Radar


In [13]:
# Definir las bandas para predecir la biomasa
bands = ['NDVI', 'MNDWI', 'NDBI', 'VV', 'VH', 'VV_plus_VH', 'VV_minus_VH', 'VV_div_VH', 'VV_VH_div_2', 'RVI', 'elevation', 'slope', 'label', 'CH']


# # Función de corrección con el ángulo de incidencia
# def apply_incidence_angle_correction(image):
#     angle = image.select('angle')
#     sigma0 = image.select(['VV', 'VH', 'HH', 'HV'])
#     correction_factor = ee.Image.constant(10).pow(angle.divide(ee.Image.constant(90)).log10())
#     corrected_sigma0 = sigma0.divide(correction_factor)
#     return image.addBands(corrected_sigma0, None, True)


# Función de preprocesamiento
def preprocess(image):
    # Aplicar corrección de ángulo de incidencia
    # image = apply_incidence_angle_correction(image)
    # Filtrado de Speckle
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')

    return speckle_filtered
# Definir la función de máscara de borde
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# Filtrar y procesar la colección Sentinel-1 para VV
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate(start, end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    
    .map(mask_edge)
    .map(preprocess)
)

# Filtrar y procesar la colección Sentinel-1 para VH
img_vh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate(start, end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')
    
    .map(mask_edge)
    .map(preprocess)
)


# Promedio de imágenes VV y VH
s1_vv = img_vv.median().rename('VV')
s1_vh = img_vh.median().rename('VH')



# Calcular el índice RVI
rvi = s1_vh.divide(s1_vv.add(s1_vh)).multiply(4).rename('RVI')
vv_plus_vh = s1_vv.add(s1_vh).rename('VV_plus_VH')
vv_minus_vh = s1_vv.subtract(s1_vh).rename('VV_minus_VH')
vv_div_vh = s1_vv.divide(s1_vh).rename('VV_div_VH')
vv_vh_div_2 = s1_vv.add(s1_vh).divide(2).rename('VV_VH_div_2')

# Combinar las bandas VV, VH y RVI en una sola imagen
s1_combined = ee.Image.cat([s1_vv, s1_vh, vv_plus_vh, vv_minus_vh, vv_div_vh, vv_vh_div_2, rvi]).clip(roi)

# Añadir las bandas VV, VH, RVI y álgebra de bandas a la imagen
image = image.addBands(s1_combined)

# Añadir SRTM DEM y pendiente
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

# Añadir las bandas DEM y pendiente a la imagen
image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)

# Seleccionar solo las bandas especificadas
image = image.select(bands)

In [14]:
# Parámetros de visualización para imágenes multiespectrales en falso color nir-swir1-swir2
visFalseColor = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B11', 'B12']
}

# Parámetros de visualización para la biomasa
visBiomass = {'min': 0, 'max': 100, 'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']}

# Parámetros de visualización para Sentinel-1
visParamsS1 = {
    'min': -25,
    'max': 0,
    'bands': ['VV']
}
# Visualización de DEM y pendiente
visParamsDEM = {
    'min': 0,
    'max': 3000,
    'palette': ['000000', 'ffffff']
}

visParamsSlope = {
    'min': 0,
    'max': 45,
    'palette': ['00ff00', 'ff0000']
}


Map.centerObject(roi, 10)
# Map.addLayer(masked_image, visFalseColor, 'Sentinel-2 False Color', False)
# Map.addLayer(masked_image.select('NDVI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI', False)
# Map.addLayer(masked_image.select('MNDWI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'MNDWI', False)
# Map.addLayer(dem, visParamsDEM, 'DEM', False)
# Map.addLayer(slope, visParamsSlope, 'Slope', False)
# Map.addLayer(s1_combined, visParamsS1, 'Sentinel-1 VV & VH', False)


Map.addLayer(image.select('MNDWI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'MNDWI', False)
Map.addLayer(image.select('VV'),visParamsS1 , 'VV', False)
Map

Map(bottom=886154.0, center=[26.908228936539853, 86.13595004394384], controls=(WidgetControl(options=['positio…

In [34]:
def reproject_to_4326(image, scale=100):
    """
    Reproyecta todas las bandas de una imagen a EPSG:4326.
    
    :param image: ee.Image - La imagen de entrada
    :param scale: int - La escala deseada en metros (por defecto 100)
    :return: ee.Image - La imagen con todas las bandas reproyectadas a EPSG:4326
    """
    
    # Definir la proyección objetivo
    target_projection = ee.Projection('EPSG:4326').atScale(scale)
    
    # Obtener los nombres de las bandas
    band_names = image.bandNames()
    
    # Función para reproyectar una banda
    def reproject_band(band_name):
        return image.select(band_name).reproject(crs=target_projection)
    
    # Aplicar la reproyección a todas las bandas
    reprojected_bands = band_names.map(lambda name: reproject_band(ee.String(name)))
    
    # Combinar las bandas reproyectadas en una nueva imagen
    reprojected_image = ee.ImageCollection(reprojected_bands).toBands().rename(band_names)
    
    return reprojected_image

reprojected_image = reproject_to_4326(image)


In [37]:
def reduce_resolution_to_100m(image):
    """
    Reduce la resolución de todas las bandas de una imagen a 100 metros.
    
    :param image: ee.Image - La imagen de entrada
    :return: ee.Image - La imagen con todas las bandas reducidas a 100m de resolución
    """
    
    # Definir la proyección objetivo
    target_projection = ee.Projection('EPSG:4326').atScale(100)
    
    # Obtener los nombres de las bandas
    band_names = image.bandNames()
    
    # Función para reducir la resolución de una banda
    def reduce_band_resolution(band_name):
        band = image.select(band_name)
        reduced_band = (
            band
            .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024)
            .reproject(crs=target_projection)
        )
        return reduced_band
    
    # Aplicar la reducción de resolución a todas las bandas
    reduced_bands = band_names.map(lambda name: reduce_band_resolution(ee.String(name)))
    
    # Combinar las bandas reducidas en una nueva imagen
    reduced_image = ee.ImageCollection(reduced_bands).toBands().rename(band_names)
    
    return reduced_image


resampled_image = reduce_resolution_to_100m(reprojected_image)

# Map.addLayer(resampled_image, visParamsS1, 'imageNDVI', False)
# Map

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.0008983152841195215, 0, 0, 0, 0.0008983152841195215, 0]}, {'id': 'MNDWI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.0008983152841195215, 0, 0, 0, 0.0008983152841195215, 0]}, {'id': 'NDBI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.0008983152841195215, 0, 0, 0, 0.0008983152841195215, 0]}, {'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [0.0008983152841195215, 0, 0, 0, 0.0008983152841195215, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [0.0008983152841195215, 0, 0, 0, 0.0008983152841195215, 0]}, {'id': 'VV_plus_VH', 'data_type': {'type': 'PixelType',

In [11]:
def print_band_info(image):
    """
    Imprime la proyección y escala de cada banda en la imagen.
    
    :param image: ee.Image - La imagen de entrada
    """
    
    # Obtener los nombres de las bandas
    band_names = image.bandNames().getInfo()
    
    print("Información de las bandas:")
    print("---------------------------")
    
    for band_name in band_names:
        # Seleccionar la banda
        band = image.select(band_name)
        
        # Obtener la proyección de la banda
        projection = band.projection()
        
        # Obtener la escala nominal de la banda
        scale = band.projection().nominalScale()
        
        # Imprimir la información
        print(f"Banda: {band_name}")
        print(f"  Proyección: {projection.crs().getInfo()}")
        print(f"  Escala nominal: {scale.getInfo()} metros")
        print("---------------------------")
        
print_band_info(image)

Información de las bandas:
---------------------------
Banda: NDVI
  Proyección: EPSG:4326
  Escala nominal: 111319.49079327357 metros
---------------------------
Banda: MNDWI
  Proyección: EPSG:4326
  Escala nominal: 111319.49079327357 metros
---------------------------
Banda: NDBI
  Proyección: EPSG:4326
  Escala nominal: 111319.49079327357 metros
---------------------------
Banda: VV
  Proyección: EPSG:4326


KeyboardInterrupt: 

# Muestreo de datos

En esta seccion se genera la logica de llamado a los datos de GEDI y recoleccion de datos sobre sus pixeles, la exportacion del csv se gerena por partes para cada uno de los buffers de puntos creados anteriormente.  

In [19]:
# Filtrar los datos de biomasa de GEDI para agbd
gediData_agbd = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .map(lambda image: image.updateMask(image.select('degrade_flag').eq(0)
                                        .And(image.select('l2_quality_flag'))
                                        .And(image.select('l4_quality_flag')))) \
    .select('agbd') \
    .median() \
    .toFloat() \
    .clip(roi)

# Filtrar los datos de biomasa de GEDI para agbd_t_se
gediData_agbd_t_se = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .map(lambda image: image.updateMask(image.select('degrade_flag').eq(0)
                                        .And(image.select('l2_quality_flag'))
                                        .And(image.select('l4_quality_flag')))) \
    .select('agbd_se') \
    .median() \
    .toFloat() \
    .clip(roi)

# Convertir todas las bandas a float en image
image = image.addBands(gediData_agbd_t_se)

# Convertir la imagen a float si es necesario
def convert_to_float(image):
    return image.float()

image = convert_to_float(image)

Map.addLayer(gediData_agbd, {'palette': ['black', 'green']}, 'agbd Mask', False)
Map.addLayer(gediData_agbd_t_se, {'min': 0, 'max': 0.1, 'palette': ['blue']}, 'agbd_t_se', False)

 

In [24]:
def split_featurecollection(fc, num_splits):

    bbox = fc.geometry().bounds()
    coords = bbox.coordinates().getInfo()[0]
    xmin, ymin = coords[0]
    xmax, ymax = coords[2]

    x_step = (xmax - xmin) / num_splits
    y_step = (ymax - ymin) / num_splits

    subregions = []
    for i in range(num_splits):
        for j in range(num_splits):
            x1 = xmin + i * x_step
            x2 = xmin + (i + 1) * x_step
            y1 = ymin + j * y_step
            y2 = ymin + (j + 1) * y_step

            subregion = ee.Geometry.Rectangle([x1, y1, x2, y2])
            subregions.append(subregion)

    return subregions

num_splits = 10  # Ajusta según tus necesidades

# Divide la región en subregiones
subregions = split_featurecollection(roi, num_splits)

# Exportar los datos en lotes
for index, region in enumerate(subregions):
    sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
        scale=20,
        region=region,
        geometries=True
    )

    export_task = ee.batch.Export.table.toDrive(
        collection=sample,
        description=f'ExportSampleToCSV_{index}',
        folder='EarthEngine',
        fileNamePrefix=f'Datos_RF_{index}',
        fileFormat='CSV'
    )

    export_task.start()

    # Esperar a que la tarea de exportación se complete
    while export_task.active():
        print(f'Exportación {index} en progreso...')
        time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación {index} completada con éxito.')
    else:
        print(f'Error en la exportación {index}: {export_task.status()}')

# Visualizar las subregiones en el mapa
Map = geemap.Map()

# Añadir la FeatureCollection original al mapa
Map.addLayer(roi, {}, 'Feature Collection')

# Añadir las subregiones al mapa
for i, region in enumerate(subregions):
    Map.addLayer(ee.Feature(region), {}, f'Subregion {i + 1}')

# Centrar el mapa en la región de interés
Map.centerObject(roi, 10)

# Mostrar el mapa
Map

EEException: Request had insufficient authentication scopes.

In [17]:
import time
# Función para exportar cada buffer individualmente
def export_buffer(buffer, index):
    sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
        scale=20,
        region=buffer.geometry(),
        geometries=True
    )

    export_task = ee.batch.Export.table.toDrive(
        collection=sample,
        description=f'ExportSampleToCSV_{index}',
        folder='EarthEngine',
        fileNamePrefix=f'Datos_RF_{index}',
        fileFormat='CSV'
    )

    # Iniciar la tarea de exportación
    export_task.start()

    # Esperar a que la tarea de exportación se complete
    export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
    while export_task.active():
        print('Exportación en progreso...')
        time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación {index} completada con éxito.')
    else:
        print(f'Error en la exportación {index}: {export_task.status()}')

# Ejemplo de uso
buffered_points_list = buffered_points.toList(buffered_points.size())

for i in range(buffered_points.size().getInfo()):
    buffer = ee.Feature(buffered_points_list.get(i))
    export_buffer(buffer, i)

Exportación en progreso...
Exportación en progreso...
Exportación 0 completada con éxito.
Exportación en progreso...
Error en la exportación 1: {'state': 'FAILED', 'description': 'ExportSampleToCSV_1', 'priority': 100, 'creation_timestamp_ms': 1720553487486, 'update_timestamp_ms': 1720553515125, 'start_timestamp_ms': 1720553491213, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'error_message': 'User memory limit exceeded.', 'id': 'AU4HYUVN5PPPUPWGFZXWTGBD', 'name': 'projects/ee-facuboladerasgee/operations/AU4HYUVN5PPPUPWGFZXWTGBD'}
Exportación en progreso...
Exportación 2 completada con éxito.
Exportación en progreso...
Exportación en progreso...
Exportación 3 completada con éxito.
Exportación en progreso...
Exportación en progreso...
Exportación 4 completada con éxito.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación 5 completada con éxito.
Exportación en progreso...
Exportación en progreso...
Exportación 6 completada con éxito.
Exportación 

# Modelo

In [29]:
feature_names = ['B11', 'B12', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI', 'RVI', 'VH', 'VV', 'VV_VH_div_2', 'VV_div_VH', 'VV_minus_VH', 'VV_plus_VH', 'elevation', 'slope']
label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_Gee"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)


In [32]:
# Reducir la imagen clasificada a su valor máximo en la región de interés
max_value = classified.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=roi,
    scale=30,
    maxPixels=1e9
)

# Obtener el valor máximo de la reducción
max_agbd = max_value.get('classification').getInfo()
print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 54.255305009999994


In [31]:

# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 60,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",
)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…